In [2]:
pip install pandas yfinance matplotlib

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 10.6 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached yfinance-0.2.26-py2.py3-none-any.whl (62 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 10.2 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.3/502.3 kB 8.7 MB/s eta 0:00:0031m12.9 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 10.0 MB/s eta 0:00:00m eta 0:00:010:01:01
  Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 11.0 MB/s eta 0:00:00m eta 0:00:010:01:01
  Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 9.5 MB/s eta 0:00

In [ ]:
import pandas as pd
import yfinance as yf
from statsmodels.tsa.arima_model import ARIMA
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta

In [ ]:
def moving_average(symbol):
    # Mendapatkan data historis saham
    data = yf.download(symbol, start='2022-01-01', end='2023-12-31')

    # Menghitung SMA
    data['SMA_50'] = data['Close'].rolling(window=50).mean()
    data['SMA_100'] = data['Close'].rolling(window=100).mean()
    data['SMA_200'] = data['Close'].rolling(window=200).mean()

    # Mendapatkan harga penutupan terakhir dan SMA terakhir
    last_close = data['Close'][-1]
    last_sma_50 = data['SMA_50'][-1]
    last_sma_100 = data['SMA_100'][-1]
    last_sma_200 = data['SMA_200'][-1]

    if last_close > last_sma_50 > last_sma_100 > last_sma_200:
        return 'Saran: Beli. Harga saham di atas semua SMA.'
    elif last_close < last_sma_50 < last_sma_100 < last_sma_200:
        return 'Saran: Jual. Harga saham di bawah semua SMA.'
    else:
        return 'Saran: Tahan. Harga saham di antara SMA.'

In [ ]:
def calculate_rsi(data, window):
    delta = data.diff()
    up_days = delta.copy()
    up_days[delta<=0]=0.0
    down_days = abs(delta.copy())
    down_days[delta>0]=0.0
    RS_up = up_days.rolling(window).mean()
    RS_down = down_days.rolling(window).mean()
    rsi= 100-100/(1+RS_up/RS_down)
    return rsi

In [ ]:
def analyze_stock_rsi(symbol):
    # Mendapatkan data historis saham
    data = yf.download(symbol, start='2022-01-01', end='2023-12-31')

    # Menghitung RSI
    data['RSI'] = calculate_rsi(data['Close'], 14)

    # Mendapatkan RSI terakhir
    last_rsi = data['RSI'][-1]

    # Memberikan saran berdasarkan RSI
    if last_rsi > 70:
        return 'Saran: Jual. Saham mungkin overbought.'
    elif last_rsi < 30:
        return 'Saran: Beli. Saham mungkin oversold.'
    else:
        return 'Saran: Tahan. Saham tidak overbought atau oversold.'


In [ ]:
def calculate_macd(data, short_window, long_window):
    short_ema = data.ewm(span=short_window, adjust=False).mean()
    long_ema = data.ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=9, adjust=False).mean()
    return macd, signal

def analyze_stock_macd(symbol):
    # Mendapatkan data historis saham
    data = yf.download(symbol, start='2022-01-01', end='2023-12-31')

    # Menghitung MACD
    data['MACD'], data['Signal'] = calculate_macd(data['Close'], 12, 26)

    # Mendapatkan MACD dan Signal terakhir
    last_macd = data['MACD'][-1]
    last_signal = data['Signal'][-1]

    # Memberikan saran berdasarkan MACD dan Signal
    if last_macd > last_signal:
        return 'Saran: Beli. MACD di atas Signal.'
    elif last_macd < last_signal:
        return 'Saran: Jual. MACD di bawah Signal.'
    else:
        return 'Saran: Tahan. MACD sama dengan Signal.'

# Contoh penggunaan fungsi
print(analyze_stock_macd('BBCA'))

In [ ]:
def calculate_bollinger_bands(data, window):
    sma = data.rolling(window).mean()
    std = data.rolling(window).std()
    bollinger_up = sma + (2 * std)
    bollinger_down = sma - (2 * std)
    return bollinger_up, bollinger_down

def analyze_stock_bollinger(symbol):
    # Mendapatkan data historis saham
    data = yf.download(symbol, start='2022-01-01', end='2023-12-31')

    # Menghitung Bollinger Bands
    data['Bollinger_Up'], data['Bollinger_Down'] = calculate_bollinger_bands(data['Close'], 20)

    # Mendapatkan Bollinger Bands dan harga penutupan terakhir
    last_close = data['Close'][-1]
    last_bollinger_up = data['Bollinger_Up'][-1]
    last_bollinger_down = data['Bollinger_Down'][-1]

    # Memberikan saran berdasarkan Bollinger Bands
    if last_close > last_bollinger_up:
        return 'Saran: Jual. Harga di atas upper Bollinger Band.'
    elif last_close < last_bollinger_down:
        return 'Saran: Beli. Harga di bawah lower Bollinger Band.'
    else:
        return 'Saran: Tahan. Harga di antara Bollinger Bands.'

# Contoh penggunaan fungsi
print(analyze_stock_bollinger('BBCA'))


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
def predict_and_plot(symbol):
    # Mendapatkan data historis saham
    data = yf.download(symbol, start='2020-01-01', end='2023-12-31')

    # Menggunakan hanya data 'Close'
    data = data['Close']

    # Preprocessing data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data.values.reshape(-1, 1))

    # Membuat data training
    train_data = scaled_data[:len(data)-7]

    # Membuat model
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(None, 1)))
    model.add(LSTM(units=50))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(train_data[:-1].reshape(-1, 1, 1), train_data[1:], epochs=1, batch_size=1, verbose=2)

    # Memprediksi harga untuk 7 hari ke depan
    forecast = []
    for _ in range(7):
        pred = model.predict(train_data[-1].reshape(-1, 1, 1))
        train_data = np.append(train_data, pred)
        forecast.append(pred)

    forecast = scaler.inverse_transform(np.array(forecast).reshape(-1, 1))

    # Membatasi data ke satu bulan terakhir
    one_month_ago = datetime.now() - timedelta(days=30)
    data_last_month = data[data.index > one_month_ago]

    # Membuat plot data asli
    plt.plot(data_last_month.index, data_last_month.values, color='blue', label='Data Asli')

    # Membuat plot prediksi
    forecast_index = pd.date_range(start=data_last_month.index[-1], periods=8)[1:]
    plt.plot(forecast_index, forecast, color='red', label='Prediksi')

    plt.title('Prediksi Harga Saham ' + symbol)
    plt.xlabel('Time')
    plt.ylabel('Harga Saham')
    plt.legend()
    plt.show()

In [ ]:
predict_and_plot('TLKM.JK')

In [ ]:
moving_average("BBCA")